## 下载pts文件
ubuntu终端中
```
mkdir ~/Storage/popi_landmarks
wget https://www.creatis.insa-lyon.fr/~srit/POPI/MedPhys11/bl/pts/00.pts
……
```
##　安装SimpleITK
ubuntu终端中
```
pip install SimpleITK
```
## 要求
popi中共有6个case，将其10个时相的对应点转换成影像坐标系，保存在6*1个\*.pt文件中
在生成之后
以
|—— POPI
|   |—— case1
|       |—— 00.nii
|       |—— 01.nii
|       |—— ...
|       |—— 09.nii
|       |—— case1.pt
|   |—— case2
|       |—— 00.nii
|       |—— 01.nii
|       |—— ...
|       |—— 09.nii
|       |—— case2.pt
的结构保存在服务器/data/JY/POPI中

In [2]:

# 将txt文件中的信息保存为
import numpy as np
import torch
import SimpleITK as sitk
#
'''
将ITK世界坐标转换为影像坐标。此影像坐标指：ITK Snap中显示的起始点为1的坐标
args:
    input: itk 世界坐标
    origin itk原点
    spacing itk分辨率
'''
def single_point_itk_to_pixel(itk, origin, spacing):
    x = (itk[0] -origin[0])/spacing[0]+1
    y = (itk[1] -origin[1])/spacing[1]+1
    z = (itk[2] -origin[2])/spacing[2]+1
    return [x,y,z]
def sequence_points_itk_to_pixel(itk, origin, spacing):
    itk[:,0] = (itk[:,0] -origin[0])/spacing[0]+1
    itk[:,1] = (itk[:,1] -origin[1])/spacing[1]+1
    itk[:,2] = (itk[:,2] -origin[2])/spacing[2]+1
    return itk
'''
通过SimpleITK读取图像的itk原点、itk分辨率
'''
def get_spacing_origin(path):
    img = sitk.ReadImage(path)
    spacing = img.GetSpacing()
    origin = img.GetOrigin()
    return spacing, origin

In [3]:
path = 'F:\DataSet\PKUH3\\2_Nii\\01\\0.nii'
spacing, origin = get_spacing_origin(path)
landmark_00_itk = np.genfromtxt('00.pts', dtype = np.float32) # change to 0-based indexing
landmark_00_image = sequence_points_itk_to_pixel(landmark_00_itk, spacing, origin)-1

landmarks  = {'00':landmark_00_image, '10':landmark_10_image} # 直到90
torch.save(landmarks,f'case{case}.pt')